In [14]:

from pymongo import MongoClient

import warnings
from sklearnex import patch_sklearn

patch_sklearn()
import numpy as np
import pandas as pd
from category_encoders import (BackwardDifferenceEncoder, BaseNEncoder,
                               BinaryEncoder, CatBoostEncoder, CountEncoder,
                               GLMMEncoder, HelmertEncoder, JamesSteinEncoder,
                               LeaveOneOutEncoder, MEstimateEncoder,
                               QuantileEncoder, SummaryEncoder, TargetEncoder,
                               WOEEncoder)
from sklearn import set_config
from sklearn.base import clone as model_clone
from sklearn.cluster import *
from sklearn.compose import *
from sklearn.cross_decomposition import *
from sklearn.decomposition import *
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.gaussian_process import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.multioutput import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.pipeline import *
from sklearn.preprocessing import *
from sklearn.svm import *
from sklearn.tree import *
from sklearn.utils import *
from tqdm import tqdm, trange
from xgboost import XGBClassifier, XGBRFClassifier
from BorutaShap import BorutaShap

pd.options.plotting.backend = "plotly"
pd.options.display.max_columns = 50
set_config(display="diagram")
warnings.filterwarnings("ignore")
import pickle
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns
from joblib import parallel_backend
from joblib.memory import Memory

sns.set()
from pprint import pprint
from helpers import PolynomialWrapper as PWrapper
from helpers import NestedCVWrapper as NCVWrapper
from helpers import ColumnSelectors
import sklearn


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
from helpers import DFCollection
db = DFCollection()

In [4]:
db.save_all()

Do you want to save /home/mlop3n/PycharmProjects/sdpiit/data/train?(Yes/No/Exit) Yes


/home/mlop3n/PycharmProjects/sdpiit/data/trainSaved Successfully


Do you want to save /home/mlop3n/PycharmProjects/sdpiit/data/test?(Yes/No/Exit) Yes


/home/mlop3n/PycharmProjects/sdpiit/data/testSaved Successfully


Do you want to save /home/mlop3n/PycharmProjects/sdpiit/data/data_with_ridit?(Yes/No/Exit) Yes


...

/home/mlop3n/PycharmProjects/sdpiit/data/final_dataSaved Successfully


Do you want to save /home/mlop3n/PycharmProjects/sdpiit/data/final_pred_data?(Yes/No/Exit) Yes


/home/mlop3n/PycharmProjects/sdpiit/data/final_pred_dataSaved Successfully


Do you want to save /home/mlop3n/PycharmProjects/sdpiit/data/baseline?(Yes/No/Exit) Yes


/home/mlop3n/PycharmProjects/sdpiit/data/baselineSaved Successfully


In [6]:
def __table__(rows, margin=10, columns=[]):
        """
        Return string representing table content, returns table as string and as a list of strings.
        It is okay for rows to have different sets of keys, table will show union of columns with
        missing values being empty spaces.
        :param rows: list of dictionaries as rows
        :param margin: left space padding to apply to each row, default is 0
        :param columns: extract listed columns in provided order, other columns will be ignored
        :return: table content as string and as list
        """

        def projection(cols, columns):
            return (
                [(x, cols[x]) for x in columns if x in cols]
                if columns
                else cols.items()
            )

        def row_to_string(row, columns):
            values = [
                (row[name] if name in row else "").rjust(size) for name, size in columns
            ]
            return "|%s|" % ("|".join(values))

        def header(columns):
            return "|%s|" % ("|".join([name.rjust(size) for name, size in columns]))

        def divisor(columns):
            return "+%s+" % ("+".join(["-" * size for name, size in columns]))

        data = [dict([(str(a), str(b)) for a, b in row.items()]) for row in rows]
        cols = (
            dict([(x, len(x) + 1) for row in data for x in row.keys()]) if data else {}
        )
        for row in data:
            for key in row.keys():
                cols[key] = max(cols[key], len(row[key]) + 1)
        proj = projection(
            cols, columns
        )  # extract certain columns to display (or all if not provided)
        table = (
            [divisor(proj), header(proj), divisor(proj)]
            + [row_to_string(row, proj) for row in data]
            + [divisor(proj)]
        )
        table = ["%s%s" % (" " * margin, tpl) for tpl in table] if margin > 0 else table
        table_text = "\n".join(table)
        return (table_text, table)

def tablify( rows, margin=10, columns=[]):
    """
    Print table in console for list of rows.
    """
    txt, _ = __table__(rows, margin, columns)
    # txt = txt
    print(txt)
cls_names = [
    sklearn.ensemble._weight_boosting.AdaBoostClassifier,
    sklearn.naive_bayes.BernoulliNB,
    # sklearn.naive_bayes.CategoricalNB,
    # sklearn.naive_bayes.ComplementNB,
    sklearn.tree._classes.DecisionTreeClassifier,
    sklearn.tree._classes.ExtraTreeClassifier,
    sklearn.ensemble._forest.ExtraTreesClassifier,
    sklearn.naive_bayes.GaussianNB,
    # sklearn.gaussian_process._gpc.GaussianProcessClassifier,
    sklearn.ensemble._gb.GradientBoostingClassifier,
    sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier,
    sklearn.neighbors.KNeighborsClassifier,
    sklearn.svm._classes.LinearSVC,
    sklearn.linear_model.LogisticRegression,
    # sklearn.linear_model._logistic.LogisticRegressionCV,
    # sklearn.neural_network._multilayer_perceptron.MLPClassifier,
    sklearn.naive_bayes.MultinomialNB,
    # sklearn.neighbors._nearest_centroid.NearestCentroid,
    sklearn.svm.NuSVC,
    sklearn.linear_model._passive_aggressive.PassiveAggressiveClassifier,
    sklearn.linear_model._perceptron.Perceptron,
    # sklearn.neighbors._classification.RadiusNeighborsClassifier,
    sklearn.ensemble._forest.RandomForestClassifier,
    sklearn.linear_model._ridge.RidgeClassifier,
    # sklearn.linear_model._ridge.RidgeClassifierCV,
    sklearn.linear_model._stochastic_gradient.SGDClassifier,
    sklearn.svm.SVC,
]
column_selector = ColumnSelectors()
classifiers = [f() for f in cls_names]
dtype_info = column_selector.dtype_info

ordinal = column_selector.ordinal_cols
nominal = column_selector.nominal_cols
binary = column_selector.binary_cols
ratio = column_selector.ratio_cols


final_data = pd.read_parquet("../data/final_data.parquet", engine="fastparquet")
final_pred_data = pd.read_parquet(
    "../data/final_pred_data.parquet", engine="fastparquet"
)
baseline_prediction_data = pd.read_parquet("../data/baseline.parquet")
data__ = pd.read_parquet("../data/data_with_ridit.hdfs", engine="fastparquet")
prediction_data = pd.read_parquet("../data/test.parquet", engine="fastparquet")
data = pd.read_parquet("../data/train.parquet", engine="fastparquet")



def categorise_data(data):
    ordinal_data = data.loc[:, ordinal]
    nominal_data = data.loc[:, nominal]
    binary_data = data.loc[:, binary]
    ratio_data = data.loc[:, ratio]
    return ordinal_data, nominal_data, binary_data, ratio_data


# ordinal_data, nominal_data, binary_data, ratio_data = categorise_data(final_data)
# final_data.rename(columns={"label": "target"}, inplace=True)
# data.rename(columns={"label": "target"}, inplace=True)
# baseline_prediction_data
bs = pd.concat([final_data, baseline_prediction_data], axis=0, ignore_index=True)


def gen_balanced_trained_test(data, p):
    Y = data.target
    X_2 = Y_2 = Y[Y == 2].index
    X_0 = Y_0 = Y[Y == 0].index
    X_1 = Y_1 = Y[Y == 1].index
    train_size = int(p * Y_2.shape[0])
    test_size = int((1 - p) * Y_2.shape[0])

    train_idx_2 = np.random.choice(Y_2, (train_size,))
    train_idx_1 = np.random.choice(Y_1, (train_size,))
    train_idx_0 = np.random.choice(Y_0, (train_size,))
    train_idx = np.r_[train_idx_0, train_idx_1, train_idx_2]
    # train_idx.shape

    test_idx_2 = np.random.choice(np.setdiff1d(Y_2, train_idx_2), (test_size,))
    test_idx_1 = np.random.choice(np.setdiff1d(Y_1, train_idx_1), (test_size,))
    test_idx_0 = np.random.choice(np.setdiff1d(Y_0, train_idx_0), (test_size,))
    test_idx = np.r_[test_idx_0, test_idx_1, test_idx_2]
    # test_idx.shape
    return train_idx, test_idx


def gen_nominal_maps(bs: pd.DataFrame = bs) -> tuple[defaultdict, defaultdict]:
    nominal_master_db = bs.loc[:, nominal]
    nominal_cont_map = defaultdict(dict)
    nominal_indvl_map = defaultdict(dict)
    for c in nominal:
        un = sorted(nominal_master_db[c].unique().tolist())
        n = len(un)
        new_id = list(range(n))
        nominal_indvl_map[c] = dict(zip(un, new_id))
    start = 0
    for c in nominal:
        un = sorted(nominal_master_db[c].unique().tolist())
        n = len(un)
        new_id = list(range(start, start + n))
        nominal_cont_map[c] = dict(zip(un, new_id))
        start += n
    return nominal_indvl_map, nominal_cont_map


# nominal_indvl_map, nominal_cont_map = gen_nominal_maps()
# nominal_master_db = bs.loc[:, nominal]

# nominal_master_db_indvl = nominal_master_db.copy()
# nominal_master_db_cont = nominal_master_db.copy()


# nominal_indvl_map
def nm_indvl_data_trnsform(row):
    for c in nominal:
        curr = row[c]
        row[c] = nominal_indvl_map[c][curr]
    return row


# test1_nominal = nominal_master_db_indvl.apply(nm_indvl_data_trnsform, axis=1)


def nm_cont_data_trnsform(row):
    for c in nominal:
        curr = row[c]
        row[c] = nominal_cont_map[c][curr]
    return row


# test2_nominal = nominal_master_db_cont.apply(nm_cont_data_trnsform, axis=1)


def best_n_features(n, data=pd.DataFrame(), target=pd.DataFrame):
    if (data.shape[0] > 0) and (target.shape[0] > 0):
        mic = mutual_info_classif(data, target, discrete_features=True)
    else:
        default_data = pd.read_parquet(
            "../data/final_data.parquet", engine="fastparquet"
        )
        data_X = default_data.drop(["target"], axis=1)
        data_y = default_data.target
    return ohe.get_feature_names_out()[mic.argsort()[-n:]]


# prediction_data = pd.read_pickle("../data/pred_data.pkl")
# est_ = [("cnb",CategoricalNB()),]
def iv_woe(data, target, bins=10, show_woe=False):
    """
    :params data: pandas.DataFrame
    :params target: str
    :params bins: int
    :params show_woe: bool
    :returns newDF: pandas.DataFrame, woeDF: pandas.DataFrame
    """
    # Empty Dataframe
    newDF, woeDF = pd.DataFrame(), pd.DataFrame()
    # Extract Column Names
    cols = data.columns
    # Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in "bifc") and (len(np.unique(data[ivars])) > 1000):
            binned_x = pd.qcut(data[ivars], bins, duplicates="drop")
            d0 = pd.DataFrame({"x": binned_x, "y": data[target]})
        else:
            d0 = pd.DataFrame({"x": data[ivars], "y": data[target]})
        d0 = d0.astype({"x": str})
        d = d0.groupby("x", as_index=False, dropna=False).agg({"y": ["count", "sum"]})
        d.columns = ["Cutoff", "N", "Events"]
        d["% of Events"] = np.maximum(d["Events"], 0.5) / d["Events"].sum()
        d["Non-Events"] = d["N"] - d["Events"]
        d["% of Non-Events"] = np.maximum(d["Non-Events"], 0.5) / d["Non-Events"].sum()
        d["WoE"] = np.log(d["% of Non-Events"] / d["% of Events"])
        d["IV"] = d["WoE"] * (d["% of Non-Events"] - d["% of Events"])
        d.insert(loc=0, column="Variable", value=ivars)
        print("Information value of " + ivars + " is " + str(round(d["IV"].sum(), 6)))
        temp = pd.DataFrame(
            {"Variable": [ivars], "IV": [d["IV"].sum()]}, columns=["Variable", "IV"]
        )
        newDF = pd.concat([newDF, temp], axis=0)
        woeDF = pd.concat([woeDF, d], axis=0)
        # Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF


def wf_create(cat_encoder=TargetEncoder, model=None):
    """
    :param cat_encoder: category_encoders
    :param model: scikit-learn Model
    :return pipe: sklearn.pipeline.Pipline
    Examples of model param:

    model = ComplementNB(norm=True,fit_prior=True,)
    model = MultinomialNB()
    model = LogisticRegression(n_jobs=-1, max_iter=10000,random_state=19)
    """
    _steps = []
    encoder__name = cat_encoder.__class__.__name__
    _steps.append(
        ("PW" + encoder__name, PolynomialWrapper(feature_encoder=cat_encoder))
    )
    if model is None:
        passordinal_columns
    else:
        model__name = model.__class__.__name__
        _steps.append((model__name, model))
    pipe = Pipeline(steps=_steps)
    return pipe


def plot_m_variance(d_coll: list[tuple[float, float, float, float]]):
    """
    :param d_coll: list[tuple[float, float, float, float]
    list of data points in the format
    (abcissa, mean, std, max)
    """
    sns.set()
    ddx = [x for x, y, u, r in d_coll]
    ddc = [y for x, y, u, r in d_coll]
    ddep = [y + u for x, y, u, r in d_coll]
    dden = [y - u for x, y, u, r in d_coll]
    ddem = [r for x, y, u, r in d_coll]
    plt.plot(ddx, ddc, "b", label="\u00b5")
    plt.plot(ddx, ddep, "r", label="\u03c3" + "+")
    plt.plot(ddx, dden, "g", label="\u03c3" + "-")
    plt.plot(ddx, ddem, "y", label="\u03c3" + "max")
    fig = plt.fill_between(ddx, ddep, dden, alpha=0.5)
    fig = plt.legend()
    plt.show()
    # return plt


# # print(pipe)
# # wf_create(TargetEncoder(),RandomForestClassifier())
# final_data_indvl = final_data.copy()
# final_pred_data_indvl_test = final_pred_data.copy()
# final_data_indvl.loc[:, nominal] = test1_nominal.loc[final_data.index, :]
# # final_pred_data_indvl_test.loc[:, nominal] = test1_nominal.loc[3796:, :]
# pred_data_indvl = test1_nominal.loc[3796:,nominal].copy()
# pred_data_indvl.index = final_pred_data.index
# final_pred_data_indvl_test.loc[:, nominal] = pred_data_indvl
# # final_pred_data_indvl_test

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [16]:
{final_data:1}

TypeError: unhashable type: 'DataFrame'